In [1]:
import pandas as pd
from flask import Flask, request
import json
from elasticsearch import Elasticsearch
import numpy as np
from flask_cors import CORS


In [71]:
df = pd.read_parquet('recipes.parquet')

In [72]:
df

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38.0,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo..."
1,39.0,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,None,[Soak saffron in warm milk for 5 minutes and p...
2,40.0,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00+00:00,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,None,"[Into a 1 quart Jar with tight fitting lid, pu..."
3,41.0,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00+00:00,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"[Drain the tofu, carefully squeezing out exces..."
4,42.0,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00+00:00,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,None,"[Mix everything together and bring to a boil.,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522512,541379.0,Meg's Fresh Ginger Gingerbread,2002090414,rdsxc,PT35M,PT1H,PT1H35M,2020-12-22 15:27:00+00:00,Make and share this Meg's Fresh Ginger Gingerb...,[],...,7.6,54.4,278.2,48.5,0.8,22.8,3.9,8.0,1 8x8 cake pan,[Preheat oven to 350&deg;F Grease an 8x8 cake ...
522513,541380.0,Roast Prime Rib au Poivre with Mixed Peppercorns,211566,Denver cooks,PT3H,PT30M,PT3H30M,2020-12-22 15:32:00+00:00,"White, black, green, and pink peppercorns add ...",[https://img.sndimg.com/food/image/upload/w_55...,...,71.4,433.8,766.3,3.2,0.7,0.1,117.0,8.0,1 Roast,[Position rack in center of oven and preheat t...
522514,541381.0,Kirshwasser Ice Cream,2001131545,Jonathan F.,PT3H,PT1H,PT4H,2020-12-22 15:33:00+00:00,Make and share this Kirshwasser Ice Cream reci...,[],...,72.6,470.9,192.5,33.9,0.0,17.3,12.8,6.0,None,[heat half and half and heavy cream to a simme...
522515,541382.0,Quick & Easy Asian Cucumber Salmon Rolls,2001004241,CLUBFOODY,None,PT15M,PT15M,2020-12-22 22:11:00+00:00,"Extremely quick and easy to make, these are gr...",[https://img.sndimg.com/food/image/upload/w_55...,...,0.1,2.9,100.5,0.3,0.0,0.1,2.4,NaN,20 rolls,"[In a small bowl, combine mayo and wasabi past..."


In [37]:
df.loc[df['RecipeId'] == 39]

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
1,39.0,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,None,[Soak saffron in warm milk for 5 minutes and p...


In [38]:
json.loads(df.loc[df['RecipeId'] == 39 ].iloc[0].to_json())

{'RecipeId': 39.0,
 'Name': 'Biryani',
 'AuthorId': 1567,
 'AuthorName': 'elly9812',
 'CookTime': 'PT25M',
 'PrepTime': 'PT4H',
 'TotalTime': 'PT4H25M',
 'DatePublished': 935932320000,
 'Description': 'Make and share this Biryani recipe from Food.com.',
 'Images': ['https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/picM9Mhnw.jpg',
  'https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/picHv4Ocr.jpg'],
 'RecipeCategory': 'Chicken Breast',
 'Keywords': ['Chicken Thigh & Leg',
  'Chicken',
  'Poultry',
  'Meat',
  'Asian',
  'Indian',
  'Weeknight',
  'Stove Top'],
 'RecipeIngredientQuantities': ['1',
  '4',
  '2',
  '2',
  '8',
  '1⁄4',
  '8',
  '1⁄2',
  '1',
  '1',
  '1⁄4',
  '1⁄4',
  '1⁄2',
  '1⁄4',
  '2',
  '3',
  None,
  '2',
  '1',
  '1',
  '8',
  '2',
  '1⁄3',
  '1⁄3',
  '1⁄3',
  '6'],
 'RecipeIngredientParts': ['saffron',
  'milk',
  'hot green chili peppers',
  'onions',
  'garlic',
  'clo

In [57]:
df.loc[df['RecipeId'] == 38 ].to_dict('records')[0]

{'RecipeId': 38.0,
 'Name': 'Low-Fat Berry Blue Frozen Dessert',
 'AuthorId': 1533,
 'AuthorName': 'Dancer',
 'CookTime': 'PT24H',
 'PrepTime': 'PT45M',
 'TotalTime': 'PT24H45M',
 'DatePublished': Timestamp('1999-08-09 21:46:00+0000', tz='UTC'),
 'Description': 'Make and share this Low-Fat Berry Blue Frozen Dessert recipe from Food.com.',
 'Images': array(['https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/YUeirxMLQaeE1h3v3qnM_229%20berry%20blue%20frzn%20dess.jpg',
        'https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/AFPDDHATWzQ0b1CDpDAT_255%20berry%20blue%20frzn%20dess.jpg',
        'https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/UYgf9nwMT2SGGJCuzILO_228%20berry%20blue%20frzn%20dess.jpg',
        'https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/PeBMJN2TGSaYks2759BA_20140722_202142.jpg',
  

In [7]:
df = pd.read_parquet('no_na_recipes.parquet.gzip')


In [15]:
app = Flask(__name__)
CORS(app)

app.es_client = Elasticsearch("https://localhost:9200", basic_auth=("elastic","DEq+yKIoJag7b_ZEJl4W"), ca_certs="~/http_ca.crt")
app.df = pd.read_parquet('no_na_recipes.parquet.gzip')

@app.route('/get_recipe', methods=['GET'])
def get_recipe():
    response_object = {'status':200}
    rid = int(request.args.get('rid'))
    recipe = json.loads(app.df.loc[app.df['RecipeId'] == rid ].iloc[0].to_json())
    response_object['recipe'] = recipe
    return response_object

@app.route('/get_recipes', methods=['POST'])
def get_recipes():
    response_object = {'status':200}
    recipe_id_list = request.get_json()
    recipe_list = []
    for n in recipe_id_list:
        recipe = json.loads(app.df.loc[app.df['RecipeId'] == n ].iloc[0].to_json())
        recipe_list.append(recipe)
    response_object['list'] = recipe_list
    return response_object

@app.route('/search_recipes', methods=['GET'])
def search_recipes():
    response_object = {'status':200}
    query_term = request.args.get('query')
    results = app.es_client.search(index='foodir', query={
            "script_score":{
                "query":{
                    "dis_max": {
                        "queries": [
                            { "match": { "Name": query_term }},
                            { "match": { "RecipeIngredientParts": query_term }},
                            { "match": { "RecipeInstructions": query_term }},
                        ],
                        "tie_breaker": 0.3
                    },
                },
                "script":{
                    "source": "_score * (((doc['ReviewCount'].value * doc['AggregatedRating'].value) + (3 * 32)) / (doc['ReviewCount'].value + 32 ))"
                }
            }
        }, suggest_field='Name', suggest_text=query_term, suggest_mode='missing', size=12)
    
    total_hit = results['hits']['total']['value']
    results_df = pd.DataFrame([[hit['_source'][key] for key in hit['_source']] for hit in results['hits']['hits']], columns=list(app.df.columns))
    results_df['_score'] = [hit['_score'] for hit in results['hits']['hits']]
    response_object['total_hit'] = total_hit
    response_object['results'] = results_df.to_dict('records')
    response_object['suggest'] = results['suggest']
    return response_object

if __name__ == '__main__':
    app.run(debug=False) 

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2024-03-18 23:26:40,688] ERROR in app: Exception on /get_recipes [POST]
Traceback (most recent call last):
  File "c:\Users\NOHP\anaconda3\envs\bibooTaxed\Lib\site-packages\flask\app.py", line 1463, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\NOHP\anaconda3\envs\bibooTaxed\Lib\site-packages\flask\app.py", line 872, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\NOHP\anaconda3\envs\bibooTaxed\Lib\site-packages\flask_cors\extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "c:\Users\NOHP\anaconda3\envs\bibooTaxed\Lib\site-packages\flask\app.py", line 870, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c